<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Instructions:" data-toc-modified-id="Instructions:-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Instructions:</a></span></li><li><span><a href="#Print-package-version-information" data-toc-modified-id="Print-package-version-information-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Print package version information</a></span></li><li><span><a href="#Import-packages" data-toc-modified-id="Import-packages-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Import packages</a></span></li></ul></div>

## Instructions:
1. Install `pandas` (`pip install pandas`)
1. Install the python `selenium` package (`pip install selenium`)
1. [Install ChromeDriver](https://chromedriver.chromium.org/downloads) on your machine
  - You also need Chrome (the regular application) on your machine.
  - These two need to align. Make sure the version of Chrome you have aligns with the version of Chromedriver you install.
1. Set up the variables in this next cell, then it should run through cleanly.

In [ ]:
# ID of your survey, findable in the URL
# (e.g. SV_ajh8623Khk in https://www.qualtrics.com/responses/#/surveys/SV_ajh8623Khk)
SURVEY_ID = 'SV_a8HnVrOTaSZiuCa'

# Export all your qualtrics data to a CSV, so we can easily get the Response IDs
# of your participants. Put the (full) path to the file here.
QUALTRICS_CSV_EXPORT = '../psych2/data/quiz1.csv'

# My university has a special prefix here, yours may differ
# Try changing 'ucbpsych.yul1' to just 'www' for most people
QUALTRICS_HOST = 'https://ucbpsych.yul1.qualtrics.com/'
URL_FMT = QUALTRICS_HOST + 'Q/Data/Ajax/GetSingleResponseReport?RID=%s&SID=%s'

QUALTRICS_USERNAME = 'your_username_here'
QUALTRICS_PASSWORD = 'your_password_here'

# Where does your Chrome download things to?
DOWNLOADS_FOLDER = '/Users/everettwetchler/Downloads/'

## Print package version information

In [1]:
%load_ext watermark
%watermark -a 'Everett Wetchler' -vudtzwgr -p pandas,selenium

Author: Everett Wetchler

Last updated: 2023-02-10 18:27:39PST

Python implementation: CPython
Python version       : 3.9.6
IPython version      : 7.26.0

pandas  : 1.3.1
selenium: 4.8.0

Git hash: e7ba1d4bf515186ab32f232533d2c9f4d3c9f842

Git repo: git@github.com:wetchler/teaching.git

Watermark: 2.3.1



## Import packages

In [ ]:
import json
import os
import time

import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
df = pd.read_csv(QUALTRICS_CSV_EXPORT)
# Pull ResponseIds while ignoring extraneous header rows
RIDS = [rid for rid in df['ResponseId'] if rid.startswith('R_')]
print("Found %d responses" % len(RIDS))

In [ ]:
def start_browser():
    chrome_options = webdriver.ChromeOptions()
    settings = {
       "recentDestinations": [{
            "id": "Save as PDF",
            "origin": "local",
            "account": "",
        }],
        "selectedDestinationId": "Save as PDF",
        "version": 2
    }
    prefs = {'printing.print_preview_sticky_settings.appState': json.dumps(settings)}
    chrome_options.add_experimental_option('prefs', prefs)
    chrome_options.add_argument('--kiosk-printing')
    return webdriver.Chrome(options=chrome_options)

In [ ]:
def login_qualtrics(browser):
    browser.get('https://ucbpsych.qualtrics.com')
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.ID, "UserName")))
    field = browser.find_element(By.ID, "UserName")
    field.send_keys(QUALTRICS_USERNAME)
    field = browser.find_element(By.ID, "UserPassword")
    field.send_keys(QUALTRICS_PASSWORD)
    browser.find_element(By.ID, "loginButton").click()

In [ ]:
def download_user_pdf(rid):
    url = URL_FMT % (rid, SURVEY_ID)
    browser.get(url)
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CLASS_NAME, "InputText")))
    browser.execute_script('window.print();')
    fname = '%s.pdf' % browser.title
    fname = os.path.join(DOWNLOADS_FOLDER, fname)
    new_fname = 'Survey_%s_Response_%s.pdf' % (SURVEY_ID, rid)
    new_fname = os.path.join(DOWNLOADS_FOLDER, new_fname)
    cmd = 'mv "%s" "%s"' % (fname, new_fname)
    print('Renaming file with command: [%s]' % cmd)
    assert os.system(cmd) == 0, 'Error renaming file for response %s' % rid

In [ ]:
browser = start_browser()
login_qualtrics(browser)
time.sleep(10)  # Hacky way to let it load fully
for rid in RIDS:
    print("Fetching", rid)
    download_user_pdf(rid)

browser.close()